# Caracterização de eletrofáceis por meio de inteligência artificial com abordagem supervisionada
## Reclassificação da litologia metamorfica em xisto ou filito

- **Nome do Arquivo**: tratamento_dados_metamorfica.ipynb
- **Autor**: Rafael Takeguma Goto
- **Versão**: 1.0
- **Data de Criação**: 11/07/2024
- **Data de Modificação**: 11/07/2024

### Descrição: 
Este notebook contém a etapa do trabalho referente à reclassificação da litologia METAMORFICA NAO IDE em XISTO ou FILITO.